In [2]:
import pandas as pd

In [3]:
def drop_niet_blokken(df):
    '''Unit is een combinatie van 4 cijfers als het om een overkoepelende term gaat.
    Door alleen de regels over te houden waar unit uit meer dan 4 characters bestaat,
    houden we de regels over blokken over.'''
    dataf = df.copy()
    return dataf.loc[dataf.Unit.str.len() > 4, ]

In [4]:

def get_vaste_staf_namelist(excelfile):
    namen = pd.read_excel(excelfile, sheet_name='Sheet1')

    return list(namen['Volledige naam'].values)

VASTE_STAF_NAMEN = get_vaste_staf_namelist('../Data/HSR Vaste staf 01-05-2022.XLSX')

def check_if_vaste_staf(naam, vaste_staf_namen=VASTE_STAF_NAMEN):

    return naam in vaste_staf_namen

def check_if_vaste_staf_achternaam(achternaam, vaste_staf_namen=VASTE_STAF_NAMEN):
    vaste_staf_achternamen = [naam.split(' ')[-1].lower() for naam in vaste_staf_namen]

    return achternaam.lower() in vaste_staf_achternamen

In [5]:
check_if_vaste_staf_achternaam('Ruwa')

False

In [47]:
naam_mapping = pd.read_excel('../Data/HSR Vaste staf 01-05-2022.XLSX', sheet_name='NaamMapping')

def onderwijsnaam_naar_onderzoeksnaam(naam, naam_mapping=naam_mapping):

    return naam_mapping.loc[naam_mapping.OnderwijsNaam == naam, 'Volledige naam'].values[0]


In [48]:
onderwijsnaam_naar_onderzoeksnaam('SMG Zwakhalen')

'Sandra Zwakhalen'

In [51]:
files = ['../Data/BROS HSR realisations 2019-2020, 14.5.2022-Blokcoordinatoren en planningsgroepsleden.xlsx',
        '../Data/BROS HSR realisations 2020-2021, 11.2.2022-Blokcoordinatoren en planningsgroepsleden.xlsx',
        '../Data/BROS HSR realisations 2021-2022, 14.5.2022-Blokcoordinatoren en planningsgroepsleden.xlsx']

onderwijs = pd.DataFrame()

# zet alle jaren in 1 df
for jaar, filename in zip(range(2019, 2022), files):
    onderwijs = pd.concat([onderwijs, (pd.read_excel(filename,
                                                     sheet_name='Realisations')
                                       .dropna(subset=['Unit'])
                                       .assign(Year=jaar)
                                       .pipe(drop_niet_blokken)
                                       .drop_duplicates())],
                          axis=0)
    
onderwijs['UnitYear'] = [f"{unit}_{year}" for unit, year in zip(onderwijs['Unit'], onderwijs['Year'])]
onderwijs['Prefix'] = onderwijs['Prefix'].fillna('')
onderwijs['Naam'] = (onderwijs.Initials + " " + onderwijs.Prefix + " " + onderwijs.Name).str.replace("  ", " ").str.replace("  ", " ")

# selecteer alleen vaste staf
onderwijs['is_vaste_staf'] = onderwijs.Name.apply(check_if_vaste_staf_achternaam)
onderwijs = onderwijs.loc[onderwijs.is_vaste_staf, ]

# zorg dat namen gelijk geschreven zijn als in Pure/Onderzoek
missing_before = onderwijs['Naam'].isnull().sum()
onderwijs['Naam'] = [onderwijsnaam_naar_onderzoeksnaam(naam) for naam in onderwijs['Naam'].values]
missing_after = onderwijs['Naam'].isnull().sum()
assert missing_before == missing_after

onderwijs.to_csv('../Data/onderwijs.csv', index=False)

onderwijs

,Unit,Unit name,Category,Name,Prefix,Initials,Department,Role name,Year,UnitYear,Naam,is_vaste_staf
3,GZW1021,Een Leven Lang Gezond,B,Zwakhalen,,SMG,HSR,Member Planning Group,2019,GZW1021_2019,Sandra Zwakhalen,True
4,GZW1022,Bedreigingen van Gezondheid,B,Haastregt,van,JCM,HSR,Member Planning Group,2019,GZW1022_2019,Jolanda van Haastregt,True
5,GZW1024,"Gezondheid, Voeding en Bewegen",C,Zijlstra,,GAR,HSR,Coordinator Planning Group,2019,GZW1024_2019,G.A. Rixt Zijlstra,True
6,GZW1025,Zorg(en) voor Gezondheid,C,Groothuis,,S,HSR,Coordinator Planning Group,2019,GZW1025_2019,Siebren Groothuis,True
7,GZW1025,Zorg(en) voor Gezondheid,B,Evers,,SMAA,HSR,Member Planning Group,2019,GZW1025_2019,Silvia Evers,True
...,...,...,...,...,...,...,...,...,...,...,...,...
101,HPI4010,Organizing & Managing Patient Flows,B,Raak,van,AJA,HSR,Member Planning Group,2021,HPI4010_2021,Arno van Raak,True
104,MPH0006,Global Health Economics,C,Pavlova,,MI,HSR,Coordinator Planning Group,2021,MPH0006_2021,Milena Pavlova,True
105,MPH0007,Global Health Management,C,Pavlova,,MI,HSR,Coordinator Planning Group,2021,MPH0007_2021,Milena Pavlova,True
108,HMS4802,The Enterpreneurial Healthcare Professional,C,Raak,van,AJA,HSR,Coordinator Planning Group,2021,HMS4802_2021,Arno van Raak,True


In [52]:
onderwijs.Naam.values[0]


'Sandra Zwakhalen'

In [55]:
from itertools import combinations
from collections import Counter

all_tuples = []
for unityear in onderwijs.UnitYear.unique():
    
    # get rows of that unityear
    # get staff in that unityear
    staff = onderwijs.loc[onderwijs.UnitYear == unityear, 'Naam'].values

    # get all combinations of 2 of staff in that unityear
    for c in list(combinations(staff, 2)):
        # print(unityear, c)
        all_tuples.append(c)
        
Counter(all_tuples)
        

Counter({('S Groothuis', 'SMAA Evers'): 2,
         ('S Groothuis', 'WNJ Groot'): 2,
         ('SMAA Evers', 'WNJ Groot'): 4,
         ('AMJ Elissen', 'GAR Zijlstra'): 1,
         ('N de Jong', 'BS de Boer'): 4,
         ('N de Jong', 'S Groothuis'): 3,
         ('N de Jong', 'JPH Hamers'): 3,
         ('BS de Boer', 'S Groothuis'): 3,
         ('BS de Boer', 'JPH Hamers'): 3,
         ('S Groothuis', 'JPH Hamers'): 3,
         ('ATG Paulus', 'RMWA Drost'): 3,
         ('ATG Paulus', 'AMJ Elissen'): 3,
         ('ATG Paulus', 'SMAA Evers'): 3,
         ('ATG Paulus', 'MJC Hiligsmann'): 3,
         ('RMWA Drost', 'AMJ Elissen'): 3,
         ('RMWA Drost', 'SMAA Evers'): 4,
         ('RMWA Drost', 'MJC Hiligsmann'): 6,
         ('AMJ Elissen', 'SMAA Evers'): 3,
         ('AMJ Elissen', 'MJC Hiligsmann'): 3,
         ('SMAA Evers', 'MJC Hiligsmann'): 4,
         ('JCM van Haastregt', 'MHC Bleijlevens'): 3,
         ('SMG Zwakhalen', 'MHC Bleijlevens'): 1,
         ('SMG Zwakhalen', 'JMM M

In [61]:
onderwijs[['Name', 'Naam']].groupby('Naam').first().sort_values('Name').to_csv('../Data/onderwijsnamen.csv')

In [65]:
onderwijs['Role name'].unique()

array(['Member Planning Group', 'Coordinator Planning Group',
       'Co-Coordinator Scriptie Ba-GW', 'Coordinator Scriptie Ba-GW',
       'Member Curriculum Committee DTZ (temporary)',
       'Developer (Block/Track/Elective) ', 'Coordinator Placement',
       'Coordinator Minor', 'Member Education Programme Committee',
       'Co-Coordinator Planning Group'], dtype=object)

In [69]:
for i, w in enumerate(onderwijs['Role name'].unique()):
    print(i, w)

0 Member Planning Group
1 Coordinator Planning Group
2 Co-Coordinator Scriptie Ba-GW
3 Coordinator Scriptie Ba-GW
4 Member Curriculum Committee DTZ (temporary)
5 Developer (Block/Track/Elective) 
6 Coordinator Placement
7 Coordinator Minor
8 Member Education Programme Committee
9 Co-Coordinator Planning Group


In [72]:
HSR_COLORS = ['#001C3D', '#E84E10', '#00A2DB', '#F8B296', '#F8884A', '#005370', '#ABD1FF', '#A1360B']

kleurdict = {waarde: HSR_COLORS[i] for i, waarde in enumerate(onderwijs['is_vaste_staf'].unique())}
kleurdict

{True: '#001C3D'}

In [ ]:
kleurdict = {}

